In [70]:
import pandas as pd
import numpy as np

from IPython.display import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

c:\program files (x86)\python36-32\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [71]:
df_train = pd.read_csv('train.csv', encoding='utf8', index_col='_id')
df_test = pd.read_csv('test.csv', encoding='utf8', index_col='_id')

In [77]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24712 entries, df7489733b004bbe40d3d37b34f82419 to 46d0d25dfd1def79632dd437261d0b5c
Data columns (total 21 columns):
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64

In [168]:
def preproc_data_dtc(df_input):
   
        
    df_output = df_input.copy()
    
   # df_output['default'] = df_output['default'].apply(lambda x: 0 if x == 'no' else 1)
    
   # df_output['duration_class'] = df_output['duration'].apply(lambda x: 1 if (x >= 197 and x <= 700) else 0)
    
    #df_output = df_output.drop(['month', 'day_of_week', 'contact', 'pdays', 'nr.employed'], axis = 1)

    # category code
    df_output = pd.get_dummies(df_output, columns=['job', 'marital', 'education', 'housing', 'loan', 'poutcome', 'default', 'contact', 'month', 'day_of_week'])
        
    return df_output

In [206]:
def preproc_data(df_input):
   
        
    df_output = df_input.copy()
    
    df_output['nr.employed'] = df_output['nr.employed'].apply(lambda x: 1 if x <= 5087.65 else 0)
    
    df_output['default'] = df_output['default'].apply(lambda x: 0 if x == 'no' else 1)
    
    df_output['duration_class'] = df_output['duration'].apply(lambda x: 1 if (x >= 197 and x <= 700) else 0)
    
    #df_output['duration_class'] = df_output['duration'].apply(lambda x: 1 if ((df_output['nr.employed'] == 0) & (x <= 518.5)).all() else 0)
    
    df_output = df_output.drop(['month', 'day_of_week', 'contact', 'pdays'], axis = 1)

    # category code
    df_output = pd.get_dummies(df_output, columns=['job', 'marital', 'education', 'housing', 'loan', 'poutcome'])
        
    return df_output

In [208]:
x = preproc_data(df_train).drop(['target'], axis=1)
y = df_train['target']

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=123).split(x_train, y_train)

In [181]:
dtc = DecisionTreeClassifier(max_depth=2)
dtc.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [175]:
x.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_unknown', 'loan_yes', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'default_no',
       'default_unknown', 'default_yes', 'contact_cellular',
       'contact_telephone', 'month_apr', 'month_aug', 'month_dec', 'month_jul

In [182]:
from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz
import os 
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

dot_data = StringIO()
export_graphviz(dtc, feature_names = x.columns, out_file=dot_data, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(value=graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xd3\x00\x00\x01g\x08\x06\x00\x00\x00dG\xc5\xcf\x…

In [209]:
lr = LogisticRegression(n_jobs=-1, class_weight='balanced', solver='newton-cg', multi_class='multinomial')
# lr.fit(x_train, y_train)
# roc_auc_score(y_test, lr.predict_proba(x_test)[:,1])

lr_roc_auc = cross_val_score(
    lr,
    x, 
    y,
    scoring='roc_auc',
    cv=cv
).mean()

In [211]:
lr_roc_auc

0.9292396979043651

In [212]:
lr.fit(x, y)

df_test = pd.read_csv('test.csv', encoding='utf8')
ans = lr.predict(preproc_data(df_test.drop(['_id'], axis=1)))
#ans = lr.predict_proba(preproc_data(df_test.drop(['_id'], axis=1)))[:,1]
result = pd.DataFrame()
result['_id'] = df_test['_id']
result['target'] = ans
#result.to_csv('submit.csv', index=False)

In [213]:
result[result['target'] == 1]['target'].agg('sum')

3738

In [165]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 2 columns):
_id       16476 non-null object
target    16476 non-null int64
dtypes: int64(1), object(1)
memory usage: 193.1+ KB


In [139]:
duration_target1 = pd.DataFrame(df_train[df_train['target'] == 1].groupby('duration')['target'].agg(['count']))

In [141]:
duration_target1 = duration_target1.reset_index()
f = {'duration':['median','mean'], 'count':['sum']}
duration_target1.groupby('count').agg(f)
# duration_target1 = pd.DataFrame(duration_target1.groupby('count')['duration'].agg('median'))



duration              count
        median         mean   sum
count                            
1       1040.0  1073.823671   414
2        695.0   696.963100   542
3        509.0   542.472868   387
4        388.0   416.041667   480
5        314.5   369.400000   350
6        261.0   285.052632   228
7        271.5   327.250000   168
8        247.0   264.700000    80
9        253.5   276.250000   108
10       200.0   197.000000    30

In [155]:
duration_target0 = pd.DataFrame(df_train[df_train['target'] == 0].groupby('duration')['target'].agg(['count']))
duration_target0 = duration_target0.reset_index()
duration_target0 = pd.DataFrame(duration_target0.groupby('count').agg(f))
duration_target0

duration              count
        median         mean   sum
count                            
1       1084.0  1208.971930   285
2        821.0   860.439252   214
3        733.5   754.250000   216
4        636.0   648.245902   244
5        587.0   587.343750   160
6        557.0   572.125000   192
7        545.0   542.038462   182
8        515.0   524.304348   184
9        501.0   497.842105   171
10       449.0   472.521739   230
11       439.0   460.888889    99
12       473.0   458.923077   156
13       451.5   438.500000   130
14       432.0   426.538462   182
15       390.0   399.545455   165
16       393.0   385.384615   208
17       376.0   374.307692   221
18       336.0   297.000000    90
19       357.0   335.692308   247
20       357.5   361.875000   160
21       343.5   341.125000   168
22       324.0   331.800000   110
23       318.0   317.666667   138
24       288.0   314.600000   120
25       342.0   338.800000   125
26       327.0   328.833333   156
27       303.0   300.500000   162
28       311.0   302.285714   196
29       300.0   238.750000   232
30       279.0   239.666667   180
...        ...          ...   ...
73       152.0   142.750000   584
74        89.5    89.500000   148
75        98.0    98.000000   150
76       115.0   122.000000   380
77       116.0   105.000000   231
78       112.0   115.444444   702
79       124.5   124.500000   158
80       141.0   140.750000   320
81       119.5   115.750000   324
82        89.0    87.666667   246
83       116.0   116.000000   166
84       140.0   120.666667   252
85        83.0    87.600000   425
86       108.0   108.000000   172
87        91.0    91.000000   174
88       102.0   114.000000   264
89        84.0    97.666667   534
90        83.5    83.500000   180
91       124.0   117.400000   455
92       104.0   100.000000   276
93       126.5   126.500000   186
94       127.0   118.333333   282
95       122.0   122.000000    95
96        73.5    73.500000   192
97       125.0   125.000000    97
98       111.0   111.333333   294
100       87.0    87.000000   100
102       85.0    85.000000   102
103       73.0    73.000000   103
109       90.0    90.000000   109

[102 rows x 3 columns]

In [94]:
df_train.groupby('default')['target'].agg(['count'])

,count
default,
no,19567
unknown,5144
yes,1


In [9]:
train[train['campaign'] == 0] # means all clients have at least one call from bank

,_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target


In [10]:
train['campaign'].describe() # means all clients have at least one call from bank

count    24712.000000
mean         2.580163
std          2.776757
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         43.000000
Name: campaign, dtype: float64

In [13]:
train[train['target'] == 1]['campaign'].describe() # looks like count of calls doesn't matter a lot. 75% with target = 1 has just 2 calls

count    2787.000000
mean        2.049157
std         1.617637
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max        17.000000
Name: campaign, dtype: float64

In [12]:
train['duration'].describe()

count    24712.000000
mean       258.345298
std        261.573344
min          0.000000
25%        102.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64

In [14]:
train[train['target'] == 1]['duration'].describe() # but duration means a lot as described in overview

count    2787.000000
mean      551.896304
std       404.420029
min        37.000000
25%       255.000000
50%       442.000000
75%       739.500000
max      4199.000000
Name: duration, dtype: float64

In [58]:
train[(train['previous'] == 0)].groupby('target')['_id'].agg(['count']) # looks like if there were no calls on previous campaigns, probability  for target = 1 is low

,count
target,
0,2480
1,886


In [59]:
train[(train['previous'] != 0)].groupby('target')['_id'].agg(['count']) 

,count
target,
0,2480
1,886


In [57]:
train.groupby('target')['_id'].agg(['count'])

,count
target,
0,21925
1,2787


In [64]:
# 11.28% from total poulation has target = 1
train[train['target'] == 1]['_id'].agg(['count']) /train['_id'].agg(['count']) 

count    0.112779
Name: _id, dtype: float64

In [65]:
# 88.72% for target =0
train[train['target'] == 0]['_id'].agg(['count']) /train['_id'].agg(['count']) 

count    0.887221
Name: _id, dtype: float64

In [60]:
# Target = 1  total count = 2787,  those who doesn't have personal loan and agreed for deposit 2285
train[train['target'] == 1].groupby('loan')['_id'].agg(['count'])

,count
loan,
no,2285
unknown,70
yes,432


In [66]:
# Target = 1  total count = 2787,  but for mortgage splitted almost half and half.
train[train['target'] == 1].groupby('housing')['_id'].agg(['count'])

,count
housing,
no,1213
unknown,70
yes,1504


In [69]:
# Let's check if unknown are the same people (with unknown loan and mortgage)
# They are the same
train[(train['loan'] == 'unknown') & (train['housing'] == 'unknown') & (train['target'] == 1)]['_id'].agg(['count'])

count    70
Name: _id, dtype: int64